# 2.b) (5 pts.) Visualize the motif logo using the webtool - Generate Motifs

In [ ]:
p = {
"A": [11, 13, 5, 2, 3, 3, 1, 1, 1, 9, 14],
"C": [3, 3, 8, 2, 1, 9, 1, 1, 2, 3, 3],
"G": [4, 2, 2, 1, 24, 1, 27, 27, 1, 1, 5],
"T": [12, 12, 15, 25, 2, 17, 1, 1, 26, 17, 8],
}

dna = []
for i in range(26):
    txt = ""
    for j in range(len(p["A"])):
        for k in p:
            if p[k][j] > 1:
                txt += k
                p[k][j] -= 1
                break
    print(txt)


AAAAAAGGCAA
AAACAAGGTAA
AAATGCGGTAA
AAATGCGGTAA
AACTGCGGTAA
AACTGCGGTAA
AACTGCGGTAA
AACTGCGGTAA
AACTGCGGTCA
AACTGCGGTCA
CACTGTGGTTA
CAGTGTGGTTA
GCTTGTGGTTA
GCTTGTGGTTC
GGTTGTGGTTC
TTTTGTGGTTG
TTTTGTGGTTG
TTTTGTGGTTG
TTTTGTGGTTG
TTTTGTGGTTT
TTTTGTGGTTT
TTTTGTGGTTT
TTTTGTGGTTT
TTTTGTGGTTT
TTTTGTGGTTT
TTTTTTGGTTT


# 2.c) (25 pts.) Implement RandomizedMotifSearch on Rosalind and run it on the sequences in the attached file to compute motifs for k=11.

In [ ]:
import random

# Get profile given motifs
def get_profile(motifs):
    profile = []

    # Get counts
    for i in range(len(motifs[0])):
        s = 0
        profile.append({"A": 0, "C": 0, "G": 0, "T": 0})
        for m in motifs:
            profile[-1][m[i]] += 1
            s += 1

        # Add pseudocounts
        for b in profile[-1]:
            profile[-1][b] += 1
            s += 1

        # Convert to probabilities
        for b in profile[-1]:
            profile[-1][b] /= s

    return profile

# Get the profile most probable motif
def get_profile_most_probable_motif(seq, profile):
    max_p = -1
    max_m = None
    k = len(profile)

    # Iterate over all possible motifs
    for i in range(len(seq) - k + 1):
        motif = seq[i:i+k]

        # Calculate probability of current motif
        prob = 1
        for j, b in enumerate(motif):
            prob *= profile[j][b]

        # Keep best probability
        if prob > max_p:
            max_p = prob
            max_m = motif

    return max_m

def get_motifs(profile, dna):
    motifs = []
    for seq in dna:
        motifs.append(get_profile_most_probable_motif(seq, profile))
    return motifs

def score_motifs(motifs):
    score = 0

    # Iterate over positions
    for i in range(len(motifs[0])):
        counter = {"A": 0, "C": 0, "G": 0, "T": 0}
        max_c = -1
        max_b = None

        # Iterate over motifs
        for j, m in enumerate(motifs):

            # Get count of base at position
            counter[m[i]] += 1

            # Keep highest count base
            if counter[m[i]] > max_c:
                max_c = counter[m[i]]
                max_b = m[i]

        # Sum non-max counts
        for b in counter:
            if b != max_b:
                score += counter[b]

    return score

def randomized_motif_search(dna, k, t):

    # Get inital motifs
    motifs = []
    for seq in dna:
        i = random.randint(0, len(seq) - k)
        motifs.append(seq[i:i+k])
    best_motifs = motifs

    # Iterate until score stops changing
    while True:

        # Get new profile and motifs
        profile = get_profile(motifs)
        motifs = get_motifs(profile, dna)

        # Check score
        if score_motifs(best_motifs) > score_motifs(motifs):
            best_motifs = motifs
        else:
            return best_motifs

def multi_randomized_motif_search(dna, k, t, n):

    # Iterate n times
    best_motifs = None
    for i in range(n):

        # Get motifs
        motifs = randomized_motif_search(dna, k, t)

        # Check for new best motifs
        if best_motifs is not None:
            if score_motifs(best_motifs) > score_motifs(motifs):
                best_motifs = motifs
        else:
            best_motifs = motifs

    return best_motifs

motifs = []
with open("sequences.txt", "r") as file:
    content = file.readlines()
    line = content[0].strip().split(" ")
    k = 11
    t = len(content)
    dna = [l.strip() for l in content]
    motifs = multi_randomized_motif_search(dna, k, t, 1000)
    for m in motifs:
        print(m)

ATCTGGGGATG
GTGTGCGGTTT
AACAGCCGTTA
ATCAGAGGATA
AGCAGCTGTTG
AACTACGCTTA
ATCGTGGGTTA
ATGTGCGGACA
AGCTGAGGTTT
ATATGTCGTTA
ATCTAGGGTTA
CTCTGCGGATG
ATGTGCGCTTT
CTCTGCGCGTA
CACAGCGGCTA
ATCTGCCCATA
ATCTACGCTCA
ATCCGCCGTTG
ATATGCTGTTG
CCCTGTGGTTA


# 2.d) (20 pts.) Implement MedianString algorithm on Rosalind and run it on the same sequences in (c) to compute motifs for k=11.

In [ ]:
import itertools

# Get the hamming distance of 2 sequences
def hamming(txt1, txt2):
    dis = 0
    for i in range(len(txt1)):
        if txt1[i] != txt2[i]:
            dis += 1
    return dis

# Get all possible k-mers of size i
def gen_combos(k, k_mers=None):
    return ["".join(p) for p in itertools.product(["A", "C", "G", "T"], repeat=k)]

# Get minimum distance from pattern of a k-mer in a sequence
def d_text(seq, pat):
    min_d = 2*len(seq)
    for i in range(len(seq) - len(pat) + 1):
        txt = seq[i:i+len(pat)]
        d = hamming(pat, txt)
        if d < min_d:
            min_d = d
    return min_d

# Sum all minimum distances in dna
def d_dna(dna, pat):
    sum_d = 0
    for seq in dna:
        d = d_text(seq, pat)
        sum_d += d
    return sum_d

def min_d_dna(dna, k):
    min_d = 1000000*len(dna[0])
    min_p = None

    # Iterate over every possible k-mer
    for pat in gen_combos(k):

        # Get score of pattern in dna
        d = d_dna(dna, pat)

        # Check for new best
        if d < min_d:
            min_d = d
            min_p = pat

    return min_p

k = 11
print(min_d_dna(motifs, k))

ATCTGCGGTTA
